In [3]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd 
import random
import sklearn as skl
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree._tree import TREE_LEAF
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

Using TensorFlow backend.


In [5]:
np.random.seed(1)
# Read data from file 'filename.csv' 
# (in the same directory that your python process is based)
# Control delimiters, rows, column names with read_csv (see later) 
data = pd.read_csv("protein_only.csv") 
# Preview the first 5 lines of the loaded data 
data.head()

,SAA,CRP,VCAM_1,ICAM_1,Thrombomodulin,ICAM_3,E_Selectin,P_Selectin,LBP,MMP_1,...,NT_proBNP,GH,Fibrinogen,PAPP_A_Signal,PAPP_A,vWF,tPA,D_Dimer,GDF_15,dthmi
0,23933.4,22694.6,918.4,383.2,2.24,3.796,13.920,223.812,12568.0,108.60,...,2973.34,1035.55,2932767,0.225,0.0,38614.4,4330.467689,686.05,2680.16,1
1,5643.4,2502.4,422.8,229.4,2.50,2.176,6.100,127.340,5561.4,33.25,...,647.80,439.57,2793528,0.203,0.0,13312.4,2769.321986,224.29,1124.64,1
2,56066.4,9274.6,571.8,323.2,2.70,1.564,28.116,65.968,9385.8,24.07,...,1423.27,45.90,4080213,0.221,0.0,9842.0,26241.062210,876.05,1145.36,1
3,3938.2,1133.8,249.6,186.8,2.35,1.780,11.984,98.956,1935.6,7.17,...,15.36,23.36,2388204,0.202,0.0,9017.6,13657.778170,477.76,814.88,0
4,1372.4,718.0,383.8,184.0,3.67,2.884,10.332,100.864,5651.4,29.15,...,480.40,2.85,3610737,0.190,0.0,12484.8,19483.595630,612.35,820.32,0


In [6]:

feature_names=list(data)

print(feature_names)

datamat=np.array(data)
random.shuffle(datamat)

labels=datamat[:,-1]
datamat=datamat[:,:-1]
datamat = np.nan_to_num(datamat)

['SAA', 'CRP', 'VCAM_1', 'ICAM_1', 'Thrombomodulin', 'ICAM_3', 'E_Selectin', 'P_Selectin', 'LBP', 'MMP_1', 'MMP_3', 'MMP_9', 'TIMP_1', 'bFGF', 'sFlt_1', 'PlGF', 'VEGF', 'CKMB', 'Myo_Signal', 'Myo', 'TnI_Signal', 'TnI', 'MCP_1', 'TNFa', 'IL_1b_Signal', 'IL_1b', 'IL_2_Signal', 'IL_2', 'Il_4_Signal', 'Il_4', 'IL_6', 'IL_10', 'MCSF', 'GCSF', 'IL_1a_Signal', 'IL_1a', 'IL_1ra_Signal', 'IL_1ra', 'IL_18', 'OPGN', 'Apo_A1', 'Apo_B_Signal', 'Apo_B', 'Apo_E', 'PDGF_AA', 'PDGF_AB_BB', 'RANTES', 'MPO', 'Total_PAI_1', 'sCD40L_Signal', 'sCD40L', 'TNFb_Signal', 'TNFb', 'NT_proBNP', 'GH', 'Fibrinogen', 'PAPP_A_Signal', 'PAPP_A', 'vWF', 'tPA', 'D_Dimer', 'GDF_15', 'dthmi']


In [7]:
#Split Data into Training, Validation, and Testing Sets

train_perc=0.7
test_perc=0.15
val_perc=0.15

ind1=int(datamat.shape[0]*train_perc)
ind2=ind1+int(datamat.shape[0]*val_perc)

X_train=datamat[:ind1,:]
y_train=labels[:ind1]

X_val=datamat[ind1:ind2,:]
y_val=labels[ind1:ind2]

X_test=datamat[ind2:,:]
y_test=datamat[ind2:]

#imp = SimpleImputer(missing_values=(np.nan, np.Infinity) , strategy='constant',fill_value=0.0)
#imp = imp.fit(X_train)

In [12]:
# decision tree classifier

max_depth = 4
min_samples_split = 2
model = tree.DecisionTreeClassifier(max_depth = max_depth,min_samples_split=min_samples_split)
model.fit(X_train,y_train)
#y_predict=model.predict(X_test)
err_train = 1 - model.score(X_train,y_train)
err_val =  1 - model.score(X_val,y_val)

In [13]:
print(err_train)
print(err_val)

0.12827225130890052
0.37037037037037035


In [62]:
# choose better hyperparameters (start w/ the ones we did for project)
depth_array = np.array([None,10,20,30,40,50,60])
min_samp_array = np.array([2,4,8,12,16])

In [63]:

errvec0 = np.zeros((len(depth_array),len(min_samp_array)))  # match dimensions of arrays
errvec1 = np.zeros((len(depth_array),len(min_samp_array))) 
for i1 in range(0,len(depth_array)):
    max_depth = depth_array[i1]
    for i2 in range(0,len(min_samp_array)):
        min_samples_split=min_samp_array[i2]
        model = tree.DecisionTreeClassifier(max_depth = max_depth,min_samples_split=min_samples_split)
        model.fit(X_train,y_train)
        err0 = 1 - model.score(X_train,y_train)
        err1 = 1 - model.score(X_val,y_val)
        errvec0[i1,i2]=err0
        errvec1[i1,i2]=err1

In [64]:
errvec0

array([[0.        , 0.0078534 , 0.03403141, 0.03403141, 0.04973822],
       [0.0052356 , 0.0104712 , 0.03403141, 0.03141361, 0.05235602],
       [0.        , 0.0078534 , 0.03141361, 0.03403141, 0.05235602],
       [0.        , 0.0104712 , 0.03141361, 0.03403141, 0.04973822],
       [0.        , 0.0078534 , 0.03403141, 0.03141361, 0.04973822],
       [0.        , 0.0078534 , 0.03403141, 0.03141361, 0.04973822],
       [0.        , 0.0078534 , 0.03403141, 0.03141361, 0.04973822]])

In [65]:
errvec1

array([[0.28395062, 0.30864198, 0.27160494, 0.28395062, 0.30864198],
       [0.27160494, 0.2962963 , 0.28395062, 0.28395062, 0.28395062],
       [0.28395062, 0.28395062, 0.2962963 , 0.27160494, 0.28395062],
       [0.28395062, 0.27160494, 0.28395062, 0.28395062, 0.30864198],
       [0.2962963 , 0.2962963 , 0.27160494, 0.27160494, 0.2962963 ],
       [0.28395062, 0.25925926, 0.28395062, 0.2962963 , 0.2962963 ],
       [0.30864198, 0.2962963 , 0.28395062, 0.2962963 , 0.2962963 ]])

In [36]:
# w/ depth 10, min samp 10...26% err for validation
# w/ depth 50, min samp 10...26% err
# depth 50, min samp 8
# w/ depth 10, min samp 4 ... 23% 


In [71]:
depth_array = np.array([None,10,20,30,40,50,60])
min_samp_array =4
min_samples_leaf_arr = ([1,2,4,8])

errvec0 = np.zeros((len(depth_array),len(min_samples_leaf_arr)))  # match dimensions of arrays
errvec1 = np.zeros((len(depth_array),len(min_samples_leaf_arr))) 
for i1 in range(0,len(depth_array)):
    max_depth = depth_array[i1]
    for i2 in range(0,len(min_samples_leaf_arr)):
        min_samples_leaf=min_samples_leaf_arr[i2]
        model = tree.DecisionTreeClassifier(max_depth = max_depth,
                                            min_samples_split=min_samples_split,min_samples_leaf=min_samples_leaf)
        model.fit(X_train,y_train)
        err0 = 1 - model.score(X_train,y_train)
        err1 = 1 - model.score(X_val,y_val)
        errvec0[i1,i2]=err0
        errvec1[i1,i2]=err1
print(errvec0)


[[0.04973822 0.05497382 0.07591623 0.10732984]
 [0.05235602 0.05497382 0.07591623 0.10732984]
 [0.04973822 0.05497382 0.07591623 0.10732984]
 [0.05235602 0.05235602 0.07591623 0.10732984]
 [0.05235602 0.05497382 0.07591623 0.10732984]
 [0.05235602 0.05497382 0.07591623 0.10732984]
 [0.05235602 0.05235602 0.07591623 0.10732984]]


In [72]:
print(errvec1)

[[0.32098765 0.2962963  0.33333333 0.32098765]
 [0.28395062 0.2962963  0.2962963  0.32098765]
 [0.32098765 0.30864198 0.32098765 0.32098765]
 [0.28395062 0.2962963  0.32098765 0.32098765]
 [0.28395062 0.30864198 0.2962963  0.32098765]
 [0.2962963  0.30864198 0.2962963  0.32098765]
 [0.28395062 0.2962963  0.30864198 0.32098765]]
